In [11]:
import requests
import json

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account
API_KEY = "zFkywJI8S-AIC-x7VoilGVScDC1ektGdiSZh9Y7l_AaD"

token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey": API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"messages":[{"content":"PREPARE A WELL RESEARCHED ARTICLE ABOUT WORLD WAR 2","role":"user"}]}

# The 'stream' parameter is crucial for handling the streaming response
# REPLACE THE DEPLOYMENT ID HERE
deployment_id = "83992f72-f218-457d-a5b1-098baa1006a4" # <--- REPLACE THIS PLACEHOLDER
endpoint_url = f"https://eu-gb.ml.cloud.ibm.com/ml/v4/deployments/{deployment_id}/ai_service_stream?version=2021-05-01"

response_scoring = requests.post(endpoint_url, json=payload_scoring,
                                 headers=header, stream=True)

# --- DEBUGGING LINES ARE HERE ---
print(f"API Call Status Code: {response_scoring.status_code}")
if response_scoring.status_code != 200:
    print(f"Error Message: {response_scoring.text}")
    
print("\nScoring response\n---")

# This will store all the parts of the article
full_article = ""

# Iterate over the streamed data line by line
if response_scoring.status_code == 200:
    for line in response_scoring.iter_lines():
        if line:
            # The data is in the format 'data: {json_content}'
            try:
                line = line.decode('utf-8')
                if line.startswith('data:'):
                    json_data = json.loads(line[5:].strip())
                    # Extract the generated content and append it
                    if "choices" in json_data and json_data["choices"][0]["delta"] and "content" in json_data["choices"][0]["delta"]:
                        content = json_data["choices"][0]["delta"]["content"]
                        full_article += content
            except Exception as e:
                # Handle potential parsing errors if any
                pass

# Finally, print the complete article
print(full_article)

API Call Status Code: 200

Scoring response
---
[{"title":"World War II - Wikipedia","description":"World War II is generally considered to have begun on 1 September 1939, when Nazi Germany, under Adolf Hitler, invaded Poland.","url":"https://en.wikipedia.org/wiki/World_War_II"},{"title":"World War II | Facts, Summary, History, Dates, Combatants ...","description":"World War II was a conflict that involved virtually every part of the world during 1939–45. The main combatants were the Axis powers (Germany, Italy, ...","url":"https://www.britannica.com/event/World-War-II"},{"title":"Research Starters: Worldwide Deaths in World War II | The National ...","description":"Worldwide Casualties*. Battle Deaths, 15,000,000. Battle Wounded, 25,000,000. Civilian Deaths, 45,000,000. *Worldwide casualty estimates vary widely in several ...","url":"https://www.nationalww2museum.org/students-teachers/student-resources/research-starters/research-starters-worldwide-deaths-world-war"},{"title":"DPAA Fam